## LLM News API stream

In [91]:
import requests
from pprint import pprint 
from dotenv import load_dotenv
import os
import pandas as pd

In [73]:
#get secrets from .env
load_dotenv()

API_KEY_NEWS = os.getenv('API_KEY_NEWS')
API_KEY_NEWSDATA = os.getenv('API_KEY_NEWSDATA')



In [ ]:
#Search for keyword in title
response = requests.get(f"https://newsapi.org/v2/everything?q=Gemini&searchIn=title&sortBy=popularity&from=2024-07-20&language=en&apiKey={API_KEY_NEWS}") 
#response = requests.get(f"https://newsapi.org/v2/everything?q=LLM&title=Gemini&sortBy=popularity&from=2024-07-20&language=en&apiKey={API_KEY}") 


print(response.status_code)
json_res = response.json()
pprint(json_res)

In [51]:
for x in json_res['articles']:
    print(x['title']+ ' - ' + x['source']['name'])



Google just updated Gemini with a brand-new voice - Android Police
Report: Google wants Gemini smart glasses, looking to partner with Ray-Ban’s EssilorLuxottica - 9to5google.com
Finally, new Android update gives Gemini the lock screen feature we've all been hoping for - ZDNet
Google wants to steal Ray-Ban maker from Meta for Gemini smart glasses - Android Authority
Google teases 22 Pixel 9 Pro camera, Gemini, & other features [Video] - 9to5Google - Slashdot.org


### Get news for all models names newsapi.org

In [117]:
# Function to search for any article for a query 
def search_articles_newsapi(name):
    # Define the API endpoint and API key
    url = "https://newsapi.org/v2/everything"
    params = {
        'q': f'{name}',  # Search for the name 
        'searchIn': 'title',
        'sortBy': 'popularity',
        'from': '2024-07-20',
        'language': 'en',
        'apiKey': API_KEY_NEWS
    }
    response = requests.get(url, params=params)
    return response.json()

# Function to get headlines for a query
def search_headlines_newsapi(name):
    # Define the API endpoint and API key
    url = "https://newsapi.org/v2/top-headlines"
    params = {
        'q': f'{name}',  # Search for the name 
        'language': 'en',
        'apiKey': API_KEY_NEWS
    }
    response = requests.get(url, params=params)
    return response.json()



In [ ]:
data = pd.read_csv('all_assets.csv')
names = data['name'].tolist()
names

# Loop through each name and perform the search
results = {}
for name in names[:10]:
    result = search_articles_newsapi(name)
    results[name] = result

# Print the results (or you can save them to a file)
for name, result in results.items():
    print(f"Results for {name}:")
    print(result)

In [131]:
search_headlines_newsapi('LLM')

{'status': 'ok',
 'totalResults': 1,
 'articles': [{'source': {'id': None, 'name': 'NPR'},
   'author': '',
   'title': 'As new tech threatens jobs, Silicon Valley promotes no-strings cash aid - NPR',
   'description': 'Many tech entrepreneurs have long suggested that guaranteed income could cushion job losses from AI and automation. The latest and largest study of the idea was spearheaded by the man behind ChatGPT.',
   'url': 'https://www.npr.org/2024/07/19/nx-s1-5035263/basic-income-cash-aid-ai-sam-altman-silicon-valley-jobs',
   'urlToImage': 'https://npr.brightspotcdn.com/dims3/default/strip/false/crop/5000x2813+0+0/resize/1400/quality/100/format/jpeg/?url=http%3A%2F%2Fnpr-brightspot.s3.amazonaws.com%2F93%2Fd7%2Fcaf6121546d9968582eee8d2c367%2Faibasicincome.jpg',
   'publishedAt': '2024-07-22T05:00:00Z',
   'content': "The rise of artificial intelligence has stoked fears that such technological advances will wipe out millions of jobs. Silicon Valley entrepreneurs have thought about

### NewsData.io 

In [111]:

#response = requests.get('https://newsdata.io/api/1/archive?apikey=pub_4912971fb6dd5ccd45f7a21d6a3293bf094e8&q=Gemini&language=en&from_date=2024-07-20')
# response = requests.get(F'https://newsdata.io/api/1/latest?apikey={API_KEY_NEWSDATA}&q=Llama 3&language=en&prioritydomain=medium&country=au,us,gb&category=technology')
# print(response.status_code)
# json_res = response.json()
# pprint(json_res)
# Function to search for each name

def search_articles_newsdataapi(name):
    # Define the API endpoint and API key
    url = "https://newsdata.io/api/1/latest"
    params = {
        'qInTitle':  f'{name}',
        'language': 'en',
        'prioritydomain': 'medium',
        'country': 'au,us,gb,jp',
        'category': 'technology',
        'apiKey': API_KEY_NEWSDATA
    }
    response = requests.get(url, params=params)
    # more general search
    if response.json()['totalResults'] == 0:
        params = {
        'q':  f'{name}',
        'language': 'en',
        'prioritydomain': 'medium',
        'country': 'au,us,gb',
        'category': 'technology',
        'apiKey': API_KEY_NEWSDATA
        }
        response = requests.get(url, params=params)
        
    return response.json()

In [ ]:
model_list = ['GPT-4o', 'GPT-4','GPT-3','Llama 3','Llama 2','Gemini 1.5 Flash','Gemini']
model_companies = ['OpenAI', 'Meta Llama', 'Gemini', 'GPT', 'LLM']
# Loop through each model and perform the search
results = {}
for name in model_list:
    result = search_articles_newsdataapi(name)
    results[name] = result
    print(f'Getting {name}')
for company in model_companies:
    result = search_articles_newsdataapi(company)
    results[company] = result
    print('getting ' + company)
    
# Prepare data for MongoDB insertion
documents = []
for name, result in results.items():
    articles = result.get('results', [])
    filtered_articles = []
    
    for article in articles:
        filtered_article = {
            'title': article.get('title'),
            'link': article.get('link'),
            'description': article.get('description'),
            'pubDate': article.get('pubDate'),
            'image_url': article.get('image_url'),
            'source_url': article.get('source_url'),
            'source_icon': article.get('source_icon'),
            'country': article.get('country')
        }
        filtered_articles.append(filtered_article)  
        
    doc = {
        'query': name,
        'articles': filtered_articles
    }
    documents.append(doc)

# Print the inserted documents (or save them to a file if needed)
for doc in documents:
    print(json.dumps(doc, indent=4))

# Insert the documents into MongoDB
db = client['lighthouse']

# Get collection 
collection = db['llm_news']
collection.insert_many(documents)



### Connect to Mongo

In [128]:
from pymongo import MongoClient

#get secrets from .env
load_dotenv()

uri = os.getenv('URI')

# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!
